In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Bước 1: Import Modules

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf

# Bước 2: Load bộ dữ liệu NER

Sau khi đọc tập dữ liệu, chúng ta quan sát thử 50 dòng đầu tiên như thế nào.


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Giảng dạy/NLP cho KHDL/Dataset-En/ner_dataset.csv', encoding="latin1")
data = data.fillna(method="ffill")
data.head(50)


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


#Visualizing 
Thông kê bộ dữ liệu:
- Số lượng từ duy nhất trong kho ngữ liệu. (Tập từ vựng, số chiều)
- Số lượng thẻ duy nhất trong tập dữ liệu. (Tập nhãn)


Bằng cách sử dụng hàm **“nunique”.**

In [ ]:
print("Unique words in corpus:", data['Word'].nunique())
print("Unique tags in corpus:", data['Tag'].nunique())

Unique words in corpus: 35178
Unique tags in corpus: 17


Tạo một danh sách và sử dụng phương thức set() để lấy các giá trị trong cột “word” và loại bỏ giá trị trùng.\
Thêm phần đệm có tên “Endpad”.


In [ ]:
#tạo tập từ vựng
words = list(set(data["Word"].values))
words.append("ENDPAD")
num_words = len(words)

In [ ]:
#tạo tập nhãn tương tự như trên
# tags = ?
# num_tags = ?

# Bước 3: Lấy các câu và các thẻ tương ứng

In [ ]:
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

#Yêu cầu các bạn có thể viết theo cách khác, không dùng code sẵn

In [ ]:
getter = SentenceGetter(data)
sentences = getter.sentences

In [ ]:
sentences[0]

# Bước 4: Ánh xạ giữa các câu và chỉ mục
Chúng ta có thể thấy rằng mỗi từ được gán cho một thẻ duy nhất. Chúng tôi có thể truy xuất những từ này bằng cách sử dụng các chỉ số của chúng và tra cứu chúng trong từ điển và trả về các khóa tương ứng.


In [ ]:
word2idx = {w: i   for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [ ]:
# word2idx
word2idx["ENDPAD"]
tag2idx 

{'I-geo': 0,
 'I-art': 1,
 'I-org': 2,
 'B-art': 3,
 'I-per': 4,
 'O': 5,
 'B-geo': 6,
 'B-eve': 7,
 'I-gpe': 8,
 'I-nat': 9,
 'I-eve': 10,
 'B-per': 11,
 'I-tim': 12,
 'B-gpe': 13,
 'B-tim': 14,
 'B-org': 15,
 'B-nat': 16}

#Bước 5: Padding  đầu vào và cắt dữ liệu thành tập Train/Test\
Trong trục X, có thể thấy độ dài an toàn của câu là khoảng 50.

In [ ]:
#vẽ biểu đồ thể hiện phân bố chiều dài của câu trong tập dữ liệu.

- Sử dụng hàm pad_sequence() cho padding
- X sẽ là một đại diện bằng số cho các từ. (sử dụng word2idx)
- ‘Post’ chỉ là một giá trị của đối số đệm ở cuối câu.
- X,y là vector

In [ ]:
#ánh xạ từ trong câu và nhãn về index của nó
from tensorflow.keras.preprocessing.sequence import pad_sequences
#đặt max length cho phù hợp
# max_len = ?

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=num_words-1)

#tương tự với nhãn
# y = ?


In [ ]:
print(X[0])
print(y[0])

[24230 22497  8198 27594 14432 30578 33916  4533 26182  6790 12542 12273
 14393 33425 16004  6790 24554 22497 28351 34142 21421 23160 14276  1134
 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178
 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178
 35178 35178]
[ 5  5  5  5  5  5  6  5  5  5  5  5  6  5  5  5  5  5 13  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5]


In [1]:
idx2word(35178)

NameError: ignored

- test_size = 0.2 có nghĩa là 80% dữ liệu chia cho tập huấn luyện và 20% còn lại cho tập kiểm tra.

In [ ]:
from sklearn.model_selection import train_test_split 
# x_train, x_test, y_train, y_test = ?

# Bước 6: Xây dựng Mô hình LSTM hai chiều (bi-LSTM)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

Bản tóm tắt cho thấy rằng chúng ta có hơn 1,8 triệu tham số cần được huấn luyện.

In [ ]:
model = Sequential() #Yêu cầu khỏi tạo mạng LSTM 1 chiều

#Input

#Embedding

#SpatialDropout1D

#LSTM

#TimeDistributed

model.summary()



- Sử dụng trình tối ưu hoá "adam"
- sparce_categorical_crossentropy làm hàm mất mát
- Ma trận mà chúng ta sẽ quan tâm là ma trận độ chính xác.

In [ ]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

#Bước7: Huấn luyện Model

- Để bắt đầu huấn luyện là gọi model.fit ()
- Có thể tăng kích thước batch_size nếu bạn có GPU có kích thước bộ nhớ lớn.



In [ ]:
# Các bạn nhớ là phải lưu mô hình khi train (có nhắc lại yêu cầu ở cuối) 
import time
start_time = time.time()

history = model.fit(
    x=x_train,
    y=y_train,
    # validation_data=(x_valid,y_valid),
    validation_split= 0.1,
    batch_size= 32, 
    epochs=1,
    verbose=1
)
print("[{}] Completed!".format(time.time() - start_time))

# Bước8: Đánh giá mô hình NER



In [ ]:
model.evaluate(x_test, y_test)

- Dự đoán

In [ ]:
# i = np.random.randint(0, x_test.shape[0]) #659
print(x_test[1])
i =10
p = model.predict(np.array([x_test[i]]))

p = np.argmax(p, axis=-1)
y_true = y_test[i]
print("{:15}{:5}\t {}\n".format("Word", "True", "Pred"))
print("-" *30)
for w, true, pred in zip(x_test[i], y_true, p[0]):
    print("{:15}{}\t{}".format(words[w-1], tags[true], tags[pred]))

[25812 24342 35079 18934 30839 31153 17821 34449 29354  9375  1821  8413
  7883 32385 23426 31153 33638 12489 32385  7883 34690 20806 35088 25334
   633  6715  1134 20806 35178 35178 35178 35178 35178 35178 35178 35178
 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178
 35178 35178]
1/1 [==============================] - 0s 27ms/step
Word           True 	 Pred

------------------------------
marine         O	O
XI             O	O
Beknazarov     O	O
brand          O	O
pre-emptive    O	O
Easley         B-geo	B-geo
phone          O	O
Asad           O	O
phone          O	O
sensual        B-geo	B-geo
midair         O	O
municipal      O	O
extort         O	O
Melbourne      O	O
incarceration  O	O
Disputes       O	O
1960           O	O
merriment      O	O
monument       O	O
Leftist        O	O
tearing        O	O
Ba'athists     O	O
furor          O	O
furor          O	O
furor          O	O
furor          O	O
furor          O	O
furor          O	O
furor          O	O
furor          O

In [ ]:
# hoàn thành yêu cầu trong file thực hành tương ứng cho câu 1 và 2.

# Bước 9: lưu model

Các bạn lưu lại model đã lưu và bộ trong số model